# Dataset preprocessing

The goal of this notebook is to create a preprocessed kaggle dataset out of the competition dataset.  
For now, the preprocessing will be based on [this notebook](https://www.kaggle.com/code/vonmainstein/imu-tof).  
It consists of the following steps:
-   Set the appropriate dtypes (helps with RAM usage).
-   Impute missing feature values with forward, backward and then 0 filling.
-   Split the dataset into multiple cross validation folds.
-   Standardize feature values.
-   Pad/Truncate the sequences to the same length.  

> Note:  
> - Demographics data set will be ignored for now.  

## Setup

### Imports

In [49]:
import os
import json
import warnings
from os.path import join
from tqdm.notebook import tqdm
from itertools import repeat, starmap, product

import numpy as np
import pandas as pd
from numpy import ndarray
import plotly.express as px
from numpy.linalg import norm
from pandas import DataFrame as DF
from scipy.spatial.transform import Rotation
from sklearn.model_selection import StratifiedGroupKFold
from kagglehub import whoami, competition_download, dataset_upload

### Supress performance warngings

In [50]:
warnings.filterwarnings(
    "ignore",
    message=(
        "DataFrame is highly fragmented.  This is usually the result of "
        "calling `frame.insert` many times.*"
    ),
    category=pd.errors.PerformanceWarning,
)

### Config

In [51]:
QUATERNION_COLS = ['rot_w', 'rot_x', 'rot_y', 'rot_z']
GRAVITY_WORLD = np.array([0, 0, 9.81], "float32")
RAW_ACCELRATION_COLS = ["acc_x", "acc_y", "acc_z"]
LINEAR_ACC_COLS = ["gravity_free_" + col for col in RAW_ACCELRATION_COLS]
COMPETITION_HANDLE = "cmi-detect-behavior-with-sensor-data"
CATEGORY_COLUMNS = [
    'row_id',
    'sequence_type',
    'sequence_id',
    'subject',
    'orientation',
    'behavior',
    'phase',
    'gesture',
]
META_DATA_COLUMNS = [
    'row_id',
    'sequence_type',
    'sequence_id',
    'sequence_counter',
    'subject',
    'orientation',
    'behavior',
    'phase',
    'gesture',
]
DATASET_DF_DTYPES = {
    "acc_x": "float32", "acc_y": "float32", "acc_z": "float32",
    "thm_1":"float32", "thm_2":"float32", "thm_3":"float32", "thm_4":"float32", "thm_5":"float32",
    "sequence_counter": "int32",
    **{col: "category" for col in CATEGORY_COLUMNS},
    **{f"tof_{i_1}_v{i_2}": "float32" for i_1, i_2 in product(range(1, 5), range(64))},
}
PREPROCESSED_DATASET_HANDLE = "mauroabidalcarrer/prepocessed-cmi-2025"
# The quantile of the sequences len used to pad/truncate during preprocessing
SEQUENCE_NORMED_LEN_QUANTILE = 0.95
# SAMPLING_FREQUENCY = 10 #Hz
N_FOLDS = 5
VALIDATION_FRACTION = 0.2
TARGET_NAMES = [
    "Above ear - pull hair",
    "Cheek - pinch skin",
    "Eyebrow - pull hair",
    "Eyelash - pull hair",
    "Feel around in tray and pull out an object",
    "Forehead - pull hairline",
    "Forehead - scratch",
    "Neck - pinch skin",
    "Neck - scratch",
    "Text on phone",
    "Wave hello",
    "Write name in air",
    "Write name on leg",
    "Drink from bottle/cup",
    "Pinch knee/leg skin",
    "Pull air toward your face",
    "Scratch knee/leg skin",
    "Glasses on/off"
]
EPSILON=1e-8
DELTA_ROTATION_ANGULAR_VELOCITY_COLS = ["angular_vel_x", "angular_vel_y", "angular_vel_z"]
DELTA_ROTATION_AXES_COLS = ["rotation_axis_x", "rotation_axis_y", "rotation_axis_z"]
EULER_ANGLES_COLS = ["euler_x", "euler_y", "euler_z"]

### Define function to get the feature columns
Feature columns change over time so it's better to have a function to get them than manually update a variable every time we add/remove features.

In [97]:
def get_feature_cols(df:DF) -> list[str]:
    return list(set(df.columns) - set(META_DATA_COLUMNS) - set(TARGET_NAMES))

### Load dataset
Requires to be logged in if this notebook is not running on kaggle, go to [your settings](https://www.kaggle.com/settings) to create an access token and put it in `~/.kaggle/`.

In [98]:
competition_dataset_path = competition_download(COMPETITION_HANDLE)
df = pd.read_csv(join(competition_dataset_path, "train.csv"), dtype=DATASET_DF_DTYPES)

## Data preprocessing

### Impute missing data
Perform forward, backward and then filling of all NaN sequences.

In [ ]:
# Missing ToF values are already imputed by -1 which is inconvinient since we want all missing values to be NaN.    
# So we replace them by NaN and then perform imputing.
def get_fillna_val_per_feature_col(df:DF) -> dict:
    return {col: 1.0 if col == 'rot_w' else 0 for col in get_feature_cols(df)}

def imputed_features(df:DF) -> DF:
    # Missing ToF values are already imputed by -1 which is inconvinient since we want all missing values to be NaN.    
    # So we replace them by NaN and then perform imputing.  
    tof_vals_to_nan = {col: -1.0 for col in df.columns if col.startswith("tof")}
    # fillna_val_per_col = {col: 1.0 if col == 'rot_w' else 0 for col in df.columns}

    df[get_feature_cols(df)] = (
        df
        .loc[:, get_feature_cols(df)]
        # df.replace with np.nan sets dtype to floar64 so we set it back to float32
        .replace(tof_vals_to_nan, value=np.nan)
        .astype("float32")
        .groupby(df["sequence_id"], observed=True, as_index=False)
        .ffill()
        .groupby(df["sequence_id"], observed=True, as_index=False)
        .bfill()
        # In case there are only nan in the column in the sequence
        .fillna(get_fillna_val_per_feature_col(df))
    )
    return df

df = imputed_features(df)

### Norm quaternions
This allows us to parse quaternions with `scipy.spatial.transform.Rotation`.

In [100]:
def norm_quat_rotations(df:DF) -> DF:
    df[QUATERNION_COLS] /= np.linalg.norm(df[QUATERNION_COLS], axis=1, keepdims=True)
    return df

### Linear acceleration
Remove gravity from the acceleration features.

In [101]:
def add_linear_acc_cols(df:DF) -> DF:
    # Vectorized version of https://www.kaggle.com/code/wasupandceacar/lb-0-82-5fold-single-bert-model#Dataset `remove_gravity_from_acc`
    rotations:Rotation = Rotation.from_quat(df[QUATERNION_COLS])
    gravity_sensor_frame = rotations.apply(GRAVITY_WORLD, inverse=True).astype("float32")
    df[LINEAR_ACC_COLS] = df[RAW_ACCELRATION_COLS] - gravity_sensor_frame
    return df

df = add_linear_acc_cols(df)

### Accelerations magnitudes
Add magnitue (norm) of both  raw acceleration and linear accelration.

In [102]:
def add_acc_magnitude(df:DF, acc_cols:list[str], acc_mag_col_name:str) -> DF:
    return df.assign(**{acc_mag_col_name: np.linalg.norm(df.loc[:, acc_cols], axis=1)})

df = add_acc_magnitude(df, RAW_ACCELRATION_COLS, "raw_acc_mag")
df = add_acc_magnitude(df, LINEAR_ACC_COLS, "linear_acc_mag")

### Quaternion angle magnitude
Refered to as "rotation angle" in top notebooks.

In [103]:
def add_quat_angle_mag(df:DF) -> DF:
    return df.assign(quat_rot_mag=np.arccos(df["rot_w"]) * 2)

df = add_quat_angle_mag(df)

### Angular velocity
Compute the axis of the rotation difference and its angular magnutide.  
Top notebooks scale the vector by the angular magnitude.  
I will also include the magnitude and unit axis vector in case they turn out to be more informative.   

In [104]:
def add_angular_velocity_features(df:DF) -> DF:
    rotations = Rotation.from_quat(df[QUATERNION_COLS])
    delta_rotations = rotations[1:] * rotations[:-1].inv()
    delta_rot_velocity = delta_rotations.as_rotvec()
    # Add extra line to avoid shape mismatch
    delta_rot_velocity = np.vstack((np.zeros((1, 3)), delta_rot_velocity))
    delta_rot_magnitude = norm(delta_rot_velocity, axis=1, keepdims=True)
    delta_rot_axes = delta_rot_velocity / (delta_rot_magnitude + EPSILON)
    df[DELTA_ROTATION_ANGULAR_VELOCITY_COLS] = delta_rot_velocity
    df[DELTA_ROTATION_AXES_COLS] = delta_rot_axes
    df["delta_rot_mag"] = delta_rot_magnitude.squeeze()

    return df

df = add_angular_velocity_features(df)

### Euler angles from quaternions

In [105]:
def rot_euler_angles(df:DF) -> ndarray:
    df[EULER_ANGLES_COLS] = (
        Rotation
        .from_quat(df[QUATERNION_COLS])
        .as_euler("xyz")
        .squeeze()
    )
    return df

df = rot_euler_angles(df)

### One hot encode target values.

In [106]:
one_hot_target = pd.get_dummies(df["gesture"])
df[one_hot_target.columns] = one_hot_target
df

,row_id,sequence_type,sequence_id,sequence_counter,subject,orientation,behavior,phase,gesture,acc_x,...,Neck - scratch,Text on phone,Wave hello,Write name in air,Write name on leg,Drink from bottle/cup,Pinch knee/leg skin,Pull air toward your face,Scratch knee/leg skin,Glasses on/off
0,SEQ_000007_000000,Target,SEQ_000007,0,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,6.683594,...,False,False,False,False,False,False,False,False,False,False
1,SEQ_000007_000001,Target,SEQ_000007,1,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,6.949219,...,False,False,False,False,False,False,False,False,False,False
2,SEQ_000007_000002,Target,SEQ_000007,2,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,5.722656,...,False,False,False,False,False,False,False,False,False,False
3,SEQ_000007_000003,Target,SEQ_000007,3,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,6.601562,...,False,False,False,False,False,False,False,False,False,False
4,SEQ_000007_000004,Target,SEQ_000007,4,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,5.566406,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
574940,SEQ_065531_000048,Non-Target,SEQ_065531,48,SUBJ_039498,Seated Lean Non Dom - FACE DOWN,Performs gesture,Gesture,Write name on leg,3.503906,...,False,False,False,False,True,False,False,False,False,False
574941,SEQ_065531_000049,Non-Target,SEQ_065531,49,SUBJ_039498,Seated Lean Non Dom - FACE DOWN,Performs gesture,Gesture,Write name on leg,3.773438,...,False,False,False,False,True,False,False,False,False,False
574942,SEQ_065531_000050,Non-Target,SEQ_065531,50,SUBJ_039498,Seated Lean Non Dom - FACE DOWN,Performs gesture,Gesture,Write name on leg,3.082031,...,False,False,False,False,True,False,False,False,False,False
574943,SEQ_065531_000051,Non-Target,SEQ_065531,51,SUBJ_039498,Seated Lean Non Dom - FACE DOWN,Performs gesture,Gesture,Write name on leg,3.964844,...,False,False,False,False,True,False,False,False,False,False


### ToF data aggregation.
Time of Flight columns take most of the data, let's reduce their size by aggregating by mean for each Time of Flight sensor.

In [107]:
def agg_tof_cols_per_sensor(df:DF) -> DF:
    for tof_idx in range(1, 6):
        tof_name = f"tof_{tof_idx}"
        tof_cols = [f"{tof_name}_v{v_idx}" for v_idx in range(64)]
        if any(map(lambda col: col not in df.columns, tof_cols)):
            print(f"Some (or) all ToF {tof_idx} columns are not in the df. Maybe you already ran this cell?")
            continue
        df = (
            df
            # Need to use a dict otherwise the name of the col will be "tof_preffix" instead of the value it contains
            .assign(**{tof_name:df[tof_cols].mean(axis="columns")})
            .drop(columns=tof_cols)
        )
    return df

df = agg_tof_cols_per_sensor(df)
df

,row_id,sequence_type,sequence_id,sequence_counter,subject,orientation,behavior,phase,gesture,acc_x,...,Drink from bottle/cup,Pinch knee/leg skin,Pull air toward your face,Scratch knee/leg skin,Glasses on/off,tof_1,tof_2,tof_3,tof_4,tof_5
0,SEQ_000007_000000,Target,SEQ_000007,0,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,6.683594,...,False,False,False,False,False,139.250000,117.109375,91.687500,123.359375,135.343750
1,SEQ_000007_000001,Target,SEQ_000007,1,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,6.949219,...,False,False,False,False,False,139.796875,119.671875,97.921875,124.406250,137.000000
2,SEQ_000007_000002,Target,SEQ_000007,2,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,5.722656,...,False,False,False,False,False,142.375000,128.359375,116.953125,125.687500,140.234375
3,SEQ_000007_000003,Target,SEQ_000007,3,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,6.601562,...,False,False,False,False,False,154.109375,142.093750,144.515625,149.078125,142.609375
4,SEQ_000007_000004,Target,SEQ_000007,4,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,5.566406,...,False,False,False,False,False,177.953125,149.453125,161.828125,163.765625,151.265625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
574940,SEQ_065531_000048,Non-Target,SEQ_065531,48,SUBJ_039498,Seated Lean Non Dom - FACE DOWN,Performs gesture,Gesture,Write name on leg,3.503906,...,False,False,False,False,False,68.562500,67.750000,144.437500,74.062500,52.843750
574941,SEQ_065531_000049,Non-Target,SEQ_065531,49,SUBJ_039498,Seated Lean Non Dom - FACE DOWN,Performs gesture,Gesture,Write name on leg,3.773438,...,False,False,False,False,False,70.234375,66.656250,144.000000,70.406250,54.531250
574942,SEQ_065531_000050,Non-Target,SEQ_065531,50,SUBJ_039498,Seated Lean Non Dom - FACE DOWN,Performs gesture,Gesture,Write name on leg,3.082031,...,False,False,False,False,False,66.671875,62.906250,136.906250,70.109375,57.468750
574943,SEQ_065531_000051,Non-Target,SEQ_065531,51,SUBJ_039498,Seated Lean Non Dom - FACE DOWN,Performs gesture,Gesture,Write name on leg,3.964844,...,False,False,False,False,False,67.218750,64.500000,140.531250,75.609375,54.937500


### Add derivatives w.r.t time features

In [108]:
def add_diff_features(df:DF) -> DF:
    df[[col + "_diff" for col in get_feature_cols(df)]] = (
        df
        .groupby("sequence_id", observed=True)
        [get_feature_cols(df)]
        .diff()
        .fillna(get_fillna_val_per_feature_col(df))
        .values
    )
    return df

df = add_diff_features(df)

In [109]:
len(get_feature_cols(df))

66

### Split into folds

In [110]:
df["gesture"].unique().tolist()

['Cheek - pinch skin',
 'Forehead - pull hairline',
 'Write name on leg',
 'Feel around in tray and pull out an object',
 'Neck - scratch',
 'Neck - pinch skin',
 'Eyelash - pull hair',
 'Eyebrow - pull hair',
 'Forehead - scratch',
 'Above ear - pull hair',
 'Wave hello',
 'Write name in air',
 'Text on phone',
 'Pull air toward your face',
 'Pinch knee/leg skin',
 'Scratch knee/leg skin',
 'Drink from bottle/cup',
 'Glasses on/off']

In [ ]:
def split_dataset_stratified_groupkfold(df: pd.DataFrame, target_col: str, group_col: str) -> list[tuple[pd.DataFrame, pd.DataFrame]]:
    sgkf = StratifiedGroupKFold(n_splits=N_FOLDS, shuffle=True)

    folds = []
    X = df.drop(columns=[target_col])
    y = df[target_col]
    groups = df[group_col]

    for train_idx, valid_idx in sgkf.split(X, y, groups):
        train_df = df.iloc[train_idx].copy()
        valid_df = df.iloc[valid_idx].copy()
        folds.append((train_df, valid_df))

    return folds

# Example usage:
folds = split_dataset_stratified_groupkfold(df, target_col="gesture", group_col="subject")

### Std norm
Standard scale the feature cols (should probably do something different for IMU cols).  
<!-- *Deprecated, std norm is now performed at dataset creation to avoid target leakage.*   -->

In [112]:
def std_norm_dataset(train:DF, val:DF) -> tuple[DF, DF]:
    means = train[get_feature_cols(df)].mean().astype("float32")
    stds = train[get_feature_cols(df)].std().astype("float32")
    train.loc[:, get_feature_cols(df)] = (train[get_feature_cols(df)] - means) / stds
    val.loc[:, get_feature_cols(df)] = (val[get_feature_cols(df)] - means) / stds
    return train, val

folds = list(starmap(std_norm_dataset, folds))

Normalize full dataset.

In [113]:
# Retain full dataset meta data for inference
full_dataset_meta_data = {
    "mean": df[get_feature_cols(df)].mean().astype("float32").to_dict(),
    "std": df[get_feature_cols(df)].std().astype("float32").to_dict(),
}
df.loc[:, get_feature_cols(df)] = (df[get_feature_cols(df)] - full_dataset_meta_data["mean"]) / full_dataset_meta_data['std']

/tmp/ipykernel_17519/629001182.py:6: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.63344209 0.68143673 0.74140548 ... 0.29969805 0.33057229 0.31433888]' has dtype incompatible with float32, please explicitly cast to a compatible dtype first.

/tmp/ipykernel_17519/629001182.py:6: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 0.40467845  0.43377058  0.49058256 ... -0.96320044 -1.007662
 -1.00162401]' has dtype incompatible with float32, please explicitly cast to a compatible dtype first.

/tmp/ipykernel_17519/629001182.py:6: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.92192948 0.93580851 0.7796449  ... 0.3332821  0.43304996 0.44690426]' has dtype incompatible with float32, please explicitly cast to a compatible dtype first.

/tmp/ipykernel_17519/629001182.py:6: F

Verify the mean and std of the full dataset.

In [114]:
df[get_feature_cols(df)].agg(["mean", "std"])

,quat_rot_mag_diff,rot_y_diff,tof_5_diff,gravity_free_acc_z_diff,thm_3,thm_3_diff,rotation_axis_z,thm_1_diff,thm_2_diff,rot_z_diff,...,rotation_axis_y_diff,rot_x_diff,thm_4_diff,rot_y,gravity_free_acc_y_diff,tof_3_diff,angular_vel_y,angular_vel_y_diff,raw_acc_mag_diff,rotation_axis_x
mean,9.645060e-10,-3.692246e-11,-8.028372e-11,-1.165675e-09,1.823665e-07,-3.880065e-10,2.590368e-09,1.079359e-10,1.297276e-09,-5.801730e-10,...,2.600327e-11,-2.359640e-10,5.100067e-10,-8.728876e-10,1.018720e-10,-6.040385e-10,-5.170893e-10,-1.785537e-10,1.399540e-11,1.557940e-11
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


Let's compare the train to validation mean/std skews.

In [115]:
pd.concat({
    "train": folds[0][0][get_feature_cols(df)].agg(["mean", "std"]),
    "validation": folds[0][1][get_feature_cols(df)].agg(["mean", "std"]),
})

quat_rot_mag_diff    rot_y_diff    tof_5_diff  \
train      mean      -8.396934e-10  8.319012e-11  1.921220e-11   
           std        1.000000e+00  1.000000e+00  1.000000e+00   
validation mean       8.399087e-03 -4.691051e-03 -1.539899e-03   
           std        1.027542e+00  9.968926e-01  9.713859e-01   

                 gravity_free_acc_z_diff         thm_3    thm_3_diff  \
train      mean             1.126881e-09 -9.541278e-09  1.327260e-09   
           std              1.000000e+00  1.000000e+00  1.000000e+00   
validation mean            -1.000036e-02  1.187420e-01 -5.515520e-04   
           std              1.075816e+00  5.015092e-01  9.381459e-01   

                 rotation_axis_z    thm_1_diff    thm_2_diff    rot_z_diff  \
train      mean     4.662625e-10  1.336252e-09 -1.102790e-09 -1.069041e-11   
           std      1.000000e+00  1.000000e+00  1.000000e+00  1.000000e+00   
validation mean     8.623340e-04  6.382858e-04 -2.109877e-03 -1.282891e-03   
           std      9.988498e-01  9.452561e-01  9.012590e-01  9.247457e-01   

                 ...  rotation_axis_y_diff    rot_x_diff    thm_4_diff  \
train      mean  ...          4.600722e-12 -3.250518e-10  5.249766e-10   
           std   ...          1.000000e+00  1.000000e+00  1.000000e+00   
validation mean  ...         -3.209863e-05 -1.291472e-03  5.808236e-03   
           std   ...          9.926278e-01  9.549724e-01  9.870661e-01   

                        rot_y  gravity_free_acc_y_diff    tof_3_diff  \
train      mean  5.300710e-09             1.960102e-11  3.610124e-10   
           std   1.000000e+00             1.000000e+00  1.000000e+00   
validation mean -5.103903e-02             9.911836e-04 -4.843285e-03   
           std   1.016217e+00             1.084252e+00  9.368650e-01   

                 angular_vel_y  angular_vel_y_diff  raw_acc_mag_diff  \
train      mean   1.436723e-10       -6.625125e-13      4.610853e-12   
           std    1.000000e+00        1.000000e+00      1.000000e+00   
validation mean  -5.214137e-03        1.040067e-03     -9.354840e-04   
           std    9.872053e-01        9.323741e-01      1.088176e+00   

                 rotation_axis_x  
train      mean     4.119005e-11  
           std      1.000000e+00  
validation mean     1.314881e-02  
           std      9.869612e-01  

[4 rows x 66 columns]

### Normalize sequences lengths.  
And turn the Dataframes into ndarrays.

#### Visualize histogram of sequences lengths.

Entire dataset sequences lengths.

In [116]:
px.histogram(
    (
        df
        .groupby("sequence_id", observed=True)
        .size()
    ),
    title="Sequence length frequency",
)

Second(to avoid always look at the first one) Train/validation split sequences lengths comparaison.

In [117]:
def get_set_sequences_lengths(set:DF, name:str) -> DF:
    return (
        set
        .groupby("sequence_id", observed=True)
        .size()
        .reset_index(name="length")
        .assign(set=name)
    )

full_se_lengths = pd.concat((
    get_set_sequences_lengths(folds[2][0], "Train"),
    get_set_sequences_lengths(folds[2][1], "Validation"),
))

fig = px.histogram(
    full_se_lengths,
    x="length",
    color="set",
    barmode="overlay",  # or 'group' if you want side-by-side bars
    nbins=50,           # adjust bin size if needed
    title="Sequence Length Distribution: Train vs Validation"
)

fig.update_traces(opacity=0.8)  # better visibility with overlay
fig.show()


In [118]:
for train, val in folds:
    print("train normed sequence len:", int(train.groupby("sequence_id", observed=True).size().quantile(SEQUENCE_NORMED_LEN_QUANTILE)))
    print("validation normed sequence len:", int(val.groupby("sequence_id", observed=True).size().quantile(SEQUENCE_NORMED_LEN_QUANTILE)))
    print()

train normed sequence len: 124
validation normed sequence len: 135

train normed sequence len: 126
validation normed sequence len: 127

train normed sequence len: 130
validation normed sequence len: 119

train normed sequence len: 126
validation normed sequence len: 131

train normed sequence len: 128
validation normed sequence len: 121



#### Sequence length norm implementation

In [119]:
gesture_cols = df["gesture"].unique()

def length_normed_sequence_feat_arr(sequence: DF, normed_sequence_len: int) -> ndarray:
    features = (
        sequence
        .loc[:, get_feature_cols(df)]
        .values
    )
    len_diff = abs(normed_sequence_len - len(features))
    if len(features) < normed_sequence_len:
        padded_features = np.pad(
            features,
            ((len_diff // 2 + len_diff % 2, len_diff // 2), (0, 0)),
        )
        return padded_features
    elif len(features) > normed_sequence_len:
        return features[len_diff // 2:-len_diff // 2]
    else:
        return features

def df_to_ndarrays(df:DF, normed_sequence_len:int) -> tuple[np.ndarray, np.ndarray]:
    sequence_it = df.groupby("sequence_id", observed=True, as_index=False)
    x = np.empty(
        shape=(len(sequence_it), normed_sequence_len, len(get_feature_cols(df))),
        dtype="float32"
    )
    y = np.empty(
        shape=(len(sequence_it), df["gesture"].nunique()),
        dtype="float32"
    )
    for sequence_idx, (_, sequence) in tqdm(enumerate(sequence_it), total=len(sequence_it)):
        normed_seq_feat_arr = length_normed_sequence_feat_arr(sequence, normed_sequence_len)
        x[sequence_idx] = normed_seq_feat_arr
        # Take the first value as they are(or at least should be) all the same in a single sequence
        y[sequence_idx] = sequence[gesture_cols].iloc[0].values

    return x, y

def get_normed_seq_len(dataset:DF) -> int:
    return int(
        dataset
        .groupby("sequence_id", observed=True)
        .size()
        .quantile(SEQUENCE_NORMED_LEN_QUANTILE)
    )

def fold_dfs_to_ndarrays(train:DF, validation:DF) -> tuple[ndarray, ndarray, ndarray, ndarray]:
    """
    Returns:
        (train X, train Y, validation X, validation Y)
    """
    normed_sequence_len = get_normed_seq_len(train)
    return (
        *df_to_ndarrays(train, normed_sequence_len),
        *df_to_ndarrays(validation, normed_sequence_len),
    )

folds_arrs = list(starmap(fold_dfs_to_ndarrays, folds))

  0%|          | 0/6567 [00:00<?, ?it/s]

  0%|          | 0/1584 [00:00<?, ?it/s]

  0%|          | 0/6417 [00:00<?, ?it/s]

  0%|          | 0/1734 [00:00<?, ?it/s]

  0%|          | 0/6525 [00:00<?, ?it/s]

  0%|          | 0/1626 [00:00<?, ?it/s]

  0%|          | 0/6570 [00:00<?, ?it/s]

  0%|          | 0/1581 [00:00<?, ?it/s]

  0%|          | 0/6525 [00:00<?, ?it/s]

  0%|          | 0/1626 [00:00<?, ?it/s]

In [120]:
full_dataset_sequence_length_norm = get_normed_seq_len(df)
full_x, full_y = df_to_ndarrays(df, full_dataset_sequence_length_norm)

  0%|          | 0/8151 [00:00<?, ?it/s]

## Create dataset

In [121]:
# Clean dataset directory if it already exists
! rm -rf preprocessed_dataset
# Create dataset direcory
! mkdir preprocessed_dataset
# Save folds
for fold_i, (train_x, train_y, val_x, val_y) in enumerate(folds_arrs):
    fold_dir_path = join("preprocessed_dataset", f"fold_{fold_i}")
    os.makedirs(fold_dir_path)
    # save features (X)
    np.save(join(fold_dir_path, "train_X.npy"), train_x, allow_pickle=False)
    np.save(join(fold_dir_path, "validation_X.npy"), val_x, allow_pickle=False)
    # Save targets (Y)
    np.save(join(fold_dir_path, "train_Y.npy"), train_y, allow_pickle=False)
    np.save(join(fold_dir_path, "validation_Y.npy"), val_y, allow_pickle=False)
# Save full dataset
full_dataset_dir_path = "preprocessed_dataset/full_dataset"
os.makedirs(full_dataset_dir_path)
np.save(join(full_dataset_dir_path, "X.npy"), full_x, allow_pickle=False)
np.save(join(full_dataset_dir_path, "Y.npy"), full_y, allow_pickle=False)
# Save dataset meta data
full_dataset_meta_data["target_names"] = one_hot_target.columns.to_list()
full_dataset_meta_data["pad_seq_len"] = full_dataset_sequence_length_norm
full_dataset_meta_data["feature_cols"] = get_feature_cols(df)

with open("preprocessed_dataset/full_dataset_meta_data.json", "w") as fp:
    json.dump(full_dataset_meta_data, fp, indent=4)

In [122]:
df[get_feature_cols(df)]

,quat_rot_mag_diff,rot_y_diff,tof_5_diff,gravity_free_acc_z_diff,thm_3,thm_3_diff,rotation_axis_z,thm_1_diff,thm_2_diff,rot_z_diff,...,rotation_axis_y_diff,rot_x_diff,thm_4_diff,rot_y,gravity_free_acc_y_diff,tof_3_diff,angular_vel_y,angular_vel_y_diff,raw_acc_mag_diff,rotation_axis_x
0,-0.031599,-0.003358,0.004162,0.040062,0.633442,-0.036447,-0.063881,-0.031457,-0.031901,-0.017050,...,0.000835,-0.006843,-0.022836,-0.716343,-0.006132,0.034665,-0.010010,-0.007949,-0.000488,-0.000930
1,-0.175892,0.084852,0.218816,-0.126511,0.681437,0.383583,-1.139422,0.708143,0.047365,-0.105488,...,-0.975351,0.083221,0.131625,-0.681840,0.234224,0.824043,-0.238310,-0.213256,0.062364,-0.346745
2,-1.239175,0.335354,0.423344,0.924289,0.741405,0.488376,-0.680482,1.828919,-1.451626,-0.263360,...,-0.056328,0.392540,0.267826,-0.549352,0.830128,2.444347,-1.075163,-0.760519,-0.157361,-0.772812
3,-1.308628,0.557607,0.311967,0.788105,0.584725,-1.407649,-1.219317,0.349304,-5.911228,-0.138743,...,0.460885,0.054061,0.464780,-0.329928,0.134826,3.524549,-0.665305,0.360630,0.187499,-1.227882
4,-0.627924,0.821370,1.126032,1.424118,0.081696,-4.438758,-1.501087,-2.316473,-2.711105,-0.189905,...,-0.131683,0.268517,0.831151,-0.007333,-0.810621,2.226724,-1.052999,-0.356597,0.717248,-0.603727
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
574940,0.130531,0.074762,0.334243,0.548920,0.292463,0.376693,-0.416056,0.761030,1.049440,0.079061,...,1.979492,-0.088787,0.500950,-0.759754,0.206750,0.155347,0.171270,0.428464,-0.435785,-1.386937
574941,-0.094305,0.046224,0.222866,-0.242204,0.314339,0.154999,-0.555313,0.043840,0.171812,-0.091601,...,-1.739969,-0.024561,0.103785,-0.740360,-0.027819,-0.020730,-0.136954,-0.285131,0.474626,-1.089402
574942,-0.072132,-0.007393,0.384869,0.810820,0.299698,-0.164578,0.801664,0.133883,-0.093691,-0.028745,...,-0.071204,0.000170,-0.191678,-0.741939,0.479225,-0.863526,-0.038358,0.080717,-1.127662,0.576502
574943,-0.100145,0.046512,-0.323895,-0.644663,0.330572,0.233752,-0.514518,0.584035,0.533072,-0.090870,...,0.060466,-0.021977,0.103806,-0.722432,-0.212499,0.493652,-0.138399,-0.097915,1.096312,-1.079692


## Dataset upload
Optionally upload the dataset to kaggle.

In [123]:
if input("Do you want to upload the  dataset to kaggle?[yes/no]").lower() == "yes":
    # Updaload the dataset
    dataset_upload(
        join(whoami()["username"], "prepocessed-cmi-2025"),
        "preprocessed_dataset",
        version_notes="Preprocessed Child Mind Institue 2025 competition dataset."
    )
else:
    print("Dataset has not been uploaded.")

Dataset has not been uploaded.
